# Fit Models

Models where fit for Articles 3, 6 and 8.

For each model we used features sets from 3 different stopwords options: english, legal, none. We compared the best results for each of the stopword options. 

We see that there appears to be some errors. High values for law and procedure. Lead to high values for full. Investigated bellow. 

In [2]:
#Imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import autosklearn.classification

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import json

import functions 
classif = functions.classification()

In [2]:
#Check target 
target = pd.read_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/target.csv')
print(len(target))
target.head()

8703


,id,date,2,3,5,6,7,8,9,10,...,19,34,35,37,41,46,P1,P4,P12,P7
0,HUDOC-ECHR-1999-001-58225,1999-03-25,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
1,HUDOC-ECHR-1999-001-58226,1999-03-25,-1,-1,-1,0,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
2,HUDOC-ECHR-1999-001-58227,1999-03-25,-1,-1,-1,2,-1,2,-1,-1,...,-1,-1,2,-1,2,-1,2,-1,-1,-1
3,HUDOC-ECHR-1999-001-58239,1999-04-29,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1
4,HUDOC-ECHR-1999-001-58251,1999-05-20,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1


## Fit Model

In [5]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}/article{}_{}.csv"

features = pd.read_csv(path.format('english','6','procedure'))
features.replace('nonviolation',0,inplace=True)
features.replace('violation',1,inplace=True)

features


,abraham,acceded,acceded applicant,acceded applicant request,acceded applicant request disclosed,access,access court,accordance,accordance article,accordance article convention,...,written observations rule,written procedure,written procedure article,written procedure article convention,zagrebelsky,zagrebelsky judge,zagrebelsky judge elected,zagrebelsky judge elected respect,özmen,target
0,0,0,0,0,0,0,0,3,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,4,0,0,...,0,2,0,0,0,0,0,0,2,0
2,0,0,0,0,0,0,0,3,0,0,...,0,2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,0,0,...,0,2,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,4,0,0,...,0,2,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def trainTest(features):
    """Return train test split from features"""
    #print(features['target'])
    X = features.drop(columns='target')
    X=(X-X.mean())/X.std() #Normalise
    y = features['target']
    return train_test_split(X, y, test_size=0.10, random_state=90)

def getResults(folder):
    print(folder)
    
    path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}/article{}_{}.csv"
    hyper_c = [0.0001,0.001,0.01,0.1,1,10]
    articles = ['3','6','8']
    parts = ["full","procedure","facts","circumstances","relevant","law","other"]
    
    
    results = pd.DataFrame()
    results_C = pd.DataFrame()
    results['title'] = ['article']+parts
    results_C['title'] = ['article']+parts
    


    for a in articles:
        print('ARTICLE: {}'.format(a))
        print('='*20)

        result = [a]
        result_C = [a]

        for p in parts:
            print('PART: {}'.format(p))

            features = pd.read_csv(path.format(folder,a,p))
            X_train, X_test, y_train, y_test = trainTest(features)
            X_train.fillna(value=0,inplace=True)

            max_acc = 0
            best_c = 'null'
            for C in hyper_c:
                try:
                    accuracy = k_fold(X_train, y_train,C)
                    print(C, accuracy)
                except:
                    print('Error:',C)


                if accuracy > max_acc:
                    max_acc = accuracy
                    best_c = C

            result.append(max_acc)
            result_C.append(best_c)

            print("Best accuracy for {} is {} with C = {}".format(p,best_c, max_acc))

        results[a] = result
        results_C[a] = result_C
    return results, results_C

# Get Results

#### Stopwords: english

In [4]:
folder = 'english'
results, results_C = getResults(folder)

english
ARTICLE: 3
PART: full
0.0001 0.6670575318401406
0.001 0.8178722002635046
0.01 0.7980676328502415
0.1 0.7914492753623188
1 0.7914492753623188
10 0.7914492753623188
Best accuracy for full is 0.001 with C = 0.8178722002635046
PART: procedure
0.0001 0.8154062362758016
0.001 0.8264251207729469
0.01 0.7755950812472551
0.1 0.7733684672815107
1 0.7733684672815107
10 0.7733684672815107
Best accuracy for procedure is 0.001 with C = 0.8264251207729469
PART: facts
0.0001 0.624628897672376
0.001 0.7783113746157225
0.01 0.7670443566095739
0.1 0.7624989020641195
1 0.7624989020641195
10 0.7624989020641195
Best accuracy for facts is 0.001 with C = 0.7783113746157225
PART: circumstances
0.0001 0.6593610013175231
0.001 0.7752415458937199
0.01 0.7510386473429952
0.1 0.7509903381642513
1 0.7509903381642513
10 0.7509903381642513
Best accuracy for circumstances is 0.001 with C = 0.7752415458937199
PART: relevant
0.0001 0.6115854194115063
0.001 0.6760518225735617
0.01 0.6736802810715854
0.1 0.67783267

In [5]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}.csv".format(folder)
results.to_csv(path,index = False)

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}_C.csv".format(folder)
results_C.to_csv(path,index = False)

print(results_C)
results

           title      3      6       8
0        article      3      6       8
1           full  0.001  0.001   0.001
2      procedure  0.001  0.001  0.0001
3          facts  0.001  0.001   0.001
4  circumstances  0.001  0.001   0.001
5       relevant      1   0.01   0.001
6            law  0.001  0.001  0.0001
7          other   0.01  0.001  0.0001


,title,3,6,8
0,article,3,6,8
1,full,0.817872,0.870478,0.825519
2,procedure,0.826425,0.875386,0.82363
3,facts,0.778311,0.806358,0.725677
4,circumstances,0.775242,0.812351,0.680163
5,relevant,0.698329,0.739739,0.692822
6,law,0.886287,0.950395,0.908581
7,other,0.81096,0.934599,0.878085


#### Stopwords: legal

In [6]:
folder = 'legal'
results, results_C = getResults(folder)

legal
ARTICLE: 3
PART: full
0.0001 0.6649341238471673
0.001 0.8113021519543258
0.01 0.8115480895915678
0.1 0.8047342995169083
1 0.8047342995169083
10 0.8047342995169083
Best accuracy for full is 0.01 with C = 0.8115480895915678
PART: procedure
0.0001 0.8221234079929733
0.001 0.8331422924901185
0.01 0.808939393939394
0.1 0.8044927536231883
1 0.8044927536231883
10 0.8044927536231883
Best accuracy for procedure is 0.001 with C = 0.8331422924901185
PART: facts
0.0001 0.624824330259113
0.001 0.7871014492753623
0.01 0.784925340360123
0.1 0.7692182696530522
1 0.7692182696530522
10 0.7692182696530522
Best accuracy for facts is 0.001 with C = 0.7871014492753623
PART: circumstances
0.0001 0.6616359244620115
0.001 0.7577119016249452
0.01 0.731130873956961
0.1 0.7377492314448837
1 0.7377492314448837
10 0.7377492314448837
Best accuracy for circumstances is 0.001 with C = 0.7577119016249452
PART: relevant
0.0001 0.6093631971892841
0.001 0.6736802810715855
0.01 0.6757575757575757
0.1 0.68879885814668

In [7]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}.csv".format(folder)
results.to_csv(path,index = False)

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}_C.csv".format(folder)
results_C.to_csv(path,index = False)

print(results_C)
results

           title      3      6       8
0        article      3      6       8
1           full   0.01  0.001   0.001
2      procedure  0.001  0.001  0.0001
3          facts  0.001  0.001   0.001
4  circumstances  0.001  0.001   0.001
5       relevant      1   0.01    0.01
6            law   0.01   0.01  0.0001
7          other   0.01   0.01   0.001


,title,3,6,8
0,article,3,6,8
1,full,0.811548,0.881366,0.836344
2,procedure,0.833142,0.871728,0.827978
3,facts,0.787101,0.806343,0.758107
4,circumstances,0.757712,0.807472,0.695288
5,relevant,0.709091,0.743293,0.701903
6,law,0.888366,0.947986,0.908581
7,other,0.801621,0.934644,0.880259


#### Stopwords: none

In [8]:
folder = 'none'
results, results_C = getResults(folder)

none
ARTICLE: 3
PART: full
0.0001 0.6357905138339921
0.001 0.8062670180061484
0.01 0.7844707949055775
0.1 0.7825450153711022
1 0.7825450153711022
10 0.7825450153711022
Best accuracy for full is 0.001 with C = 0.8062670180061484
PART: procedure
0.0001 0.8106609574000879
0.001 0.8355072463768115
0.01 0.8002415458937199
0.1 0.7689196310935441
1 0.7689196310935441
10 0.7689196310935441
Best accuracy for procedure is 0.001 with C = 0.8355072463768115
PART: facts
0.0001 0.635788317962231
0.001 0.7736144049187527
0.01 0.77800395256917
0.1 0.7735090030742204
1 0.7735090030742204
10 0.7735090030742204
Best accuracy for facts is 0.01 with C = 0.77800395256917
PART: circumstances
0.0001 0.6217193675889326
0.001 0.7465393061045236
0.01 0.7244488361879666
0.1 0.7222771190162496
1 0.7222771190162496
10 0.7222771190162496
Best accuracy for circumstances is 0.001 with C = 0.7465393061045236
PART: relevant
0.0001 0.6003249890206412
0.001 0.6670114185331577
0.01 0.6867193675889328
0.1 0.6892929292929293

In [9]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}.csv".format(folder)
results.to_csv(path,index = False)

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}_C.csv".format(folder)
results_C.to_csv(path,index = False)

print(results_C)
results

           title      3      6       8
0        article      3      6       8
1           full  0.001  0.001   0.001
2      procedure  0.001  0.001  0.0001
3          facts   0.01  0.001     0.1
4  circumstances  0.001  0.001   0.001
5       relevant    0.1   0.01     0.1
6            law    0.1  0.001   0.001
7          other   0.01   0.01    0.01


,title,3,6,8
0,article,3,6,8
1,full,0.806267,0.872947,0.779896
2,procedure,0.835507,0.859545,0.832328
3,facts,0.778004,0.824444,0.695381
4,circumstances,0.746539,0.803843,0.702047
5,relevant,0.689293,0.778439,0.704261
6,law,0.901697,0.955229,0.916898
7,other,0.837189,0.946692,0.899599


In [14]:
predictions = model.predict(X_test)
results = classif.results(y_test,predictions)

0.8260869565217391
[[32 10]
 [ 6 44]]
              precision    recall  f1-score   support

nonviolation       0.84      0.76      0.80        42
   violation       0.81      0.88      0.85        50

   micro avg       0.83      0.83      0.83        92
   macro avg       0.83      0.82      0.82        92
weighted avg       0.83      0.83      0.83        92



## Format

In [23]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}/article{}_{}.csv"

english = pd.read_csv(path.format('english','6','procedure'))
legal = pd.read_csv(path.format('legal','3','circumstances'))
none = pd.read_csv(path.format('none','6','procedure'))

file = legal
print(len(file))
file.head()
file.drop(columns='target')

500


,abducted,abduction,able,abscond,absence,abuse,acar,accept,accepted,access,...,zabchekov,çakıcı,ömer,ömer güngör,şah,şahin,şen,şerif,şerif avşar,şırnak
0,0,0,1,0,3,0,0,0,3,0,...,0,82,0,0,0,0,0,0,0,0
1,0,0,2,0,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,3,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,1,0,3,0,0,1,1,0,...,0,0,4,0,0,0,10,0,0,0
4,0,0,2,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,2,0,0,1,3,1,...,0,0,0,0,0,0,0,0,0,34
7,0,0,4,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,1,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
set(english.columns) == set(legal.columns)

False

In [15]:
X_train, X_test, y_train, y_test = trainTest(procedure)

In [27]:
for i,x in zip(X_train.isna().any().index, X_train.isna().any()):
    if x == True:
        print(i)

originated
originated in


In [32]:
X_train[['originated','originated in']]

,originated,originated in
497,0.0,0.0
156,0.0,0.0
678,0.0,0.0
574,0.0,0.0
221,0.0,0.0
298,0.0,0.0
783,0.0,0.0
251,0.0,0.0
430,0.0,0.0
418,0.0,0.0


In [33]:
#X_train, X_test, y_train, y_test = trainTest(procedure)
k_fold(X_train, y_train,1)

0.8365953425263838